<a href="https://colab.research.google.com/github/vfrantc/quaternion_conv/blob/main/Copy_of_train_resnet18_tsne_init.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/vfrantc/quaternion_conv.git

Cloning into 'quaternion_conv'...
remote: Enumerating objects: 124, done.
remote: Counting objects: 100% (124/124), done.
remote: Compressing objects: 100% (87/87), done.
remote: Total 124 (delta 40), reused 110 (delta 30), pack-reused 0
Receiving objects: 100% (124/124), 4.28 MiB | 3.14 MiB/s, done.
Resolving deltas: 100% (40/40), done.


In [ ]:
%cd quaternion_conv/

/content/quaternion_conv


In [ ]:
!pip install thop

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torchvision
import torchvision.transforms as transforms

In [ ]:
import os
import sys
from tqdm.notebook import tqdm
from glob import glob

In [ ]:
from models import *

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

In [ ]:
# Data
print('==> Preparing data..')
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

==> Preparing data..


In [ ]:
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

In [ ]:
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [ ]:
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [ ]:
# Model
print('==> Building model..')
net = ResNet18()
net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True

==> Building model..


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

In [ ]:
def train(epoch):
    print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        if batch_idx % 100 == 0:
          print(batch_idx, len(trainloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)' % (train_loss/(batch_idx+1), 100.*correct/total, correct, total))

In [ ]:
def test(epoch):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

            print(batch_idx, len(testloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)' % (test_loss/(batch_idx+1), 100.*correct/total, correct, total))

    # Save checkpoint.
    acc = 100.*correct/total
    if acc > best_acc:
        print('Saving..')
        state = {
            'net': net.state_dict(),
            'acc': acc,
            'epoch': epoch,
        }
        if not os.path.isdir('checkpoint'):
            os.mkdir('checkpoint')
        torch.save(state, './checkpoint/ckpt.pth')
        best_acc = acc

In [ ]:
for epoch in range(start_epoch, start_epoch+200):
    train(epoch)
    test(epoch)
    scheduler.step()

Streaming output truncated to the last 5000 lines.
92 100 Loss: 0.577 | Acc: 80.871% (7521/9300)
93 100 Loss: 0.579 | Acc: 80.830% (7598/9400)
94 100 Loss: 0.579 | Acc: 80.789% (7675/9500)
95 100 Loss: 0.577 | Acc: 80.812% (7758/9600)
96 100 Loss: 0.576 | Acc: 80.897% (7847/9700)
97 100 Loss: 0.576 | Acc: 80.878% (7926/9800)
98 100 Loss: 0.577 | Acc: 80.859% (8005/9900)
99 100 Loss: 0.577 | Acc: 80.860% (8086/10000)

Epoch: 23
0 391 Loss: 0.349 | Acc: 86.719% (111/128)
100 391 Loss: 0.371 | Acc: 87.252% (11280/12928)
200 391 Loss: 0.386 | Acc: 86.563% (22271/25728)
300 391 Loss: 0.386 | Acc: 86.633% (33378/38528)
0 100 Loss: 0.483 | Acc: 81.000% (81/100)
1 100 Loss: 0.423 | Acc: 83.000% (166/200)
2 100 Loss: 0.413 | Acc: 85.000% (255/300)
3 100 Loss: 0.423 | Acc: 84.500% (338/400)
4 100 Loss: 0.422 | Acc: 85.000% (425/500)
5 100 Loss: 0.394 | Acc: 85.833% (515/600)
6 100 Loss: 0.420 | Acc: 85.000% (595/700)
7 100 Loss: 0.439 | Acc: 84.500% (676/800)
8 100 Loss: 0.449 | Acc: 84.667% (76

# Feature extraction

In [ ]:
# Analyze features
modules = list(net.children())[:-1]
feature_net = nn.Sequential(*modules)
for p in net.parameters():
    p.requires_grad = False

In [ ]:
features = []
for batch_idx, (inputs, targets) in enumerate(testloader):
    inputs, targets = inputs.to(device), targets.to(device)
    outputs = feature_net(inputs)

    test_loss += loss.item()
    _, predicted = outputs.max(1)
    total += targets.size(0)
    correct += predicted.eq(targets).sum().item()

    print(batch_idx, len(testloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)' % (test_loss/(batch_idx+1), 100.*correct/total, correct, total))

    # Save checkpoint.
    acc = 100.*correct/total
    if acc > best_acc:
        print('Saving..')
        state = {
            'net': net.state_dict(),
            'acc': acc,
            'epoch': epoch,
        }
        if not os.path.isdir('checkpoint'):
            os.mkdir('checkpoint')
        torch.save(state, './checkpoint/ckpt.pth')
        best_acc = acc

NameError: ignored

# Feature visualization with t-sne

In [ ]:
import matplotlib.pyplot as plt
from sklearn import manifold, datasets
from time import time

In [ ]:
perplexity = 5
t0 = time()
tsne = manifold.TSNE(n_components=n_components, init="random", random_state=0, perplexity=perplexity, n_iter=300)
Y = tsne.fit_transform(X)
t1 = time()

print("circles, perplexity=%d in %.2g sec" % (perplexity, t1 - t0))
ax.set_title("Perplexity=%d" % perplexity)
ax.scatter(Y[red, 0], Y[red, 1], c="r")
ax.scatter(Y[green, 0], Y[green, 1], c="g")
ax.xaxis.set_major_formatter(NullFormatter())
ax.yaxis.set_major_formatter(NullFormatter())
ax.axis("tight")
